In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e8/sample_submission.csv
/kaggle/input/playground-series-s3e8/train.csv
/kaggle/input/playground-series-s3e8/test.csv


In [3]:
df = pd.read_csv("/kaggle/input/playground-series-s3e8/train.csv")

In [4]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [5]:
df = df.iloc[:,1:]

In [6]:
df.cut.unique()

array(['Premium', 'Very Good', 'Ideal', 'Good', 'Fair'], dtype=object)

In [7]:
df.color.unique()

array(['F', 'J', 'G', 'E', 'D', 'H', 'I'], dtype=object)

In [8]:
df.clarity.unique()

array(['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1'],
      dtype=object)

In [9]:
from sklearn.preprocessing import OneHotEncoder

In [10]:
def ohe_cols(df, li):
    # create a new DataFrame with the two categorical columns
    cat_df = df[li]
    
    # create an instance of the OneHotEncoder and fit it to the categorical data
    ohe = OneHotEncoder()
    ohe.fit(cat_df)
    
    # transform the categorical data using the OneHotEncoder
    ohe_array = ohe.transform(cat_df).toarray()
    
    # create column names for the one-hot encoded columns
    col_names = ohe.get_feature_names(li)
    
    # create a new DataFrame with the one-hot encoded data and column names
    ohe_df = pd.DataFrame(ohe_array, columns=col_names)
    
    # concatenate the original DataFrame with the new one-hot encoded DataFrame
    df_ohe = pd.concat([df, ohe_df], axis=1)
    
    # drop the original categorical columns
    df_ohe.drop(columns=li, inplace=True)
    
    return df_ohe

In [11]:
train = ohe_cols(df,["cut","color","clarity"])

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [12]:
x = train.drop("price",axis=1)
y = train.price

In [13]:
test = pd.read_csv("/kaggle/input/playground-series-s3e8/test.csv")

In [14]:
test = ohe_cols(test, ["cut","color",'clarity'])

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
x2 = test.iloc[:,1:]

In [16]:
from sklearn.linear_model import LinearRegression
model1 = LinearRegression()
model1.fit(x,y)
y_pred=model1.predict(x2)
sub = pd.DataFrame({"id":test.id,"price":y_pred})
sub.to_csv("submission.csv",index=False)

In [17]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)
x=pd.DataFrame(x)
x2 = sc.transform(x2)
x2 = pd.DataFrame(x2)
model2 = LinearRegression()
model2.fit(x,y)
y_pred2 = model2.predict(x2)
sub2 = pd.DataFrame({"id":test.id,"price":y_pred2})
sub2.to_csv("submission.csv",index=False)

In [21]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(1024, input_shape=(26,), activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='linear'))

# compile model
model.compile(loss='mae', optimizer='adam')

# train model
model.fit(x, y, epochs=100, batch_size=32, validation_split=0.2)

# make predictions
y2 = model.predict(x2)

Epoch 1/100
4840/4840 [==============================] - 30s 5ms/step - loss: 403.2952 - val_loss: 370.5473
Epoch 2/100
4840/4840 [==============================] - 27s 6ms/step - loss: 351.5324 - val_loss: 338.7542
Epoch 3/100
4840/4840 [==============================] - 27s 6ms/step - loss: 343.6881 - val_loss: 329.9365
Epoch 4/100
4840/4840 [==============================] - 28s 6ms/step - loss: 336.1696 - val_loss: 410.8000
Epoch 5/100
4840/4840 [==============================] - 26s 5ms/step - loss: 334.0702 - val_loss: 325.6369
Epoch 6/100
4840/4840 [==============================] - 28s 6ms/step - loss: 330.4970 - val_loss: 328.5944
Epoch 7/100
4840/4840 [==============================] - 25s 5ms/step - loss: 327.8345 - val_loss: 348.2997
Epoch 8/100
4840/4840 [==============================] - 25s 5ms/step - loss: 325.7485 - val_loss: 317.3442
Epoch 9/100
4840/4840 [==============================] - 24s 5ms/step - loss: 323.3508 - val_loss: 324.1716
Epoch 10/100
4840/4840 [====

NameError: name 'X' is not defined

In [22]:
pred= model.predict(x2)
pred

4033/4033 [==============================] - 8s 2ms/step


array([[ 848.2117],
       [2473.1306],
       [2212.706 ],
       ...,
       [7428.9116],
       [5095.4683],
       [4190.3765]], dtype=float32)

In [23]:
pred = [p[0] for p in pred]

In [25]:
sub = pd.DataFrame({"id":test.id,"price":pred})
sub.to_csv("submission.csv",index=False)

In [26]:
sub

,id,price
0,193573,848.211670
1,193574,2473.130615
2,193575,2212.706055
3,193576,816.888794
4,193577,5614.615234
...,...,...
129045,322618,3903.509277
129046,322619,2631.643066
129047,322620,7428.911621
129048,322621,5095.468262
